In [1]:
import sys
import win32com.client
import pandas as pd
import os
import ctypes
import time
import datetime

In [2]:
class CpStockChart:
    def __init__(self):
        self.objStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
        self.g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
        self.g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
        
    # PLUS 실행 기본 체크 함수
    def InitPlusCheck(self,g_objCpStatus):
        # 프로세스가 관리자 권한으로 실행 여부
        if ctypes.windll.shell32.IsUserAnAdmin():
            print('정상: 관리자권한으로 실행된 프로세스입니다.')
        else:
            print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
            return False

        # 연결 여부 체크
        if (g_objCpStatus.IsConnect == 0):
            print("PLUS가 정상적으로 연결되지 않음. ")
            return False

        # # 주문 관련 초기화
        # if (g_objCpTrade.TradeInit(0) != 0):
        #     print("주문 초기화 실패")
        #     return False

        return True
    
    # 차트 요청 - 기간 기준으로
    def RequestFromTo(self, code, dwm,fromDate, toDate, caller):
        print(code, fromDate, toDate)
        # 연결 여부 체크
        if self.InitPlusCheck(self.g_objCpStatus) == False:
            return False
 
        self.objStockChart.SetInputValue(0, code)  # 종목코드
        self.objStockChart.SetInputValue(1, ord('1'))  # 기간으로 받기
        self.objStockChart.SetInputValue(2, toDate)  # To 날짜
        self.objStockChart.SetInputValue(3, fromDate)  # From 날짜
        self.objStockChart.SetInputValue(5, [0, 1, 2, 8])  # 날짜,시간,시가,거래량
        self.objStockChart.SetInputValue(6, dwm)  # '차트 주기 - 분/틱
        self.objStockChart.SetInputValue(7, 1)  # 분틱차트 주기
        self.objStockChart.SetInputValue(9, ord('1'))  # 수정주가 사용
        self.objStockChart.BlockRequest()
 
        rqStatus = self.objStockChart.GetDibStatus()
        rqRet = self.objStockChart.GetDibMsg1()
        print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            exit()
 
        len = self.objStockChart.GetHeaderValue(3)
 
        caller.dates = []
        caller.opens = []
        caller.vols = []
        caller.times = []
        for i in range(len):
            caller.dates.append(self.objStockChart.GetDataValue(0, i))
            caller.times.append(self.objStockChart.GetDataValue(1, i))
            caller.opens.append(self.objStockChart.GetDataValue(2, i))
            caller.vols.append(self.objStockChart.GetDataValue(3, i))
 
        return 

In [3]:
class PastStock:
    def __init__(self):
        
        self.dates = []
        self.opens = []
        self.vols = []
        self.times = []
 
        self.objChart = CpStockChart()
        
        self.stock_list = ["유한양행","녹십자","광동제약","대웅제약","한미약품",
                "종근당","동아쏘시오홀딩스","JW홀딩스","일동제약","한미사이언스"]
        self.datapath = "../data/"
        self.init_stock = self.setStock(self.stock_list,self.datapath)
        self.stock_code = tuple(self.init_stock.values())
 
 
    def setStock(self,stock,datapath):
    
        """
        종목코드 설정하고 (stock)
        해당 경로 설정(datapath)
        이후 csv 가져와서 찾기
        """
        filepath = os.listdir(datapath)
        csv_file = [idx for idx in filepath if 'securities' in idx]
        print("loading csv file : ",csv_file)
        securities_csv = pd.DataFrame.from_csv("%s%s" % (datapath,"".join(csv_file)), encoding='CP949', index_col=0 , header=0)
        securities_csv = securities_csv.drop_duplicates(["종목명"])
        securities_codes = {}
        for i in stock:
            securities_codes[i] = "A" + "".join(list(securities_csv[securities_csv["종목명"] == i]["종목코드"]))
        print("securities_code : ",securities_codes)
    
        return securities_codes

    def save_Stock(self,toDate,fromDate):
        for i in range(len(self.stock_code)):
            self.dates = []
            self.opens = []
            self.vols = []
            self.times = []
            self.objChart.RequestFromTo(self.stock_code[i],ord('m'),toDate,fromDate,self )
            self.dates.reverse()
            self.times.reverse()
            self.opens.reverse()
            self.vols.reverse()
            chartData = {
                '일자' : self.dates,
                '시간' : self.times,
                '현재가' : self.opens,
                '거래량' : self.vols,
            }
            df = pd.DataFrame(chartData, columns=['일자','시간','현재가','거래량'])
            
            if toDate == "20180101":   
                df.to_csv("%s%s_%s.csv" % (self.datapath,self.stock_code[i],self.stock_list[i]), header = True, encoding="CP949")
            else:
                df.to_csv("%s%s_%s.csv" % (self.datapath,self.stock_code[i],self.stock_list[i]), mode='a', header = False, encoding="CP949")
        time.sleep(60)
        return 

In [4]:
if __name__=="__main__":
    print(sys.version)
    start = PastStock()
    now = datetime.datetime.now()
    nowDate = now.strftime("%m%d")
    timeToSave = [x for x in range(101, int(nowDate), 3) if x%100 < 32 ]
    timeToSave = [x for x in timeToSave if x%100!=0]
    timeToSave = [x for x in timeToSave if x!=230]
    for i in range(len(timeToSave)-1):
        start.save_Stock(str(20180000+int(timeToSave[i])),str(20180000+int(timeToSave[i+1])-1))

3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]


com_error: (-2147221164, '클래스가 등록되지 않았습니다.', None, None)

In [7]:
timeToSave = [x for x in range(101, 1020, 3) if x%100 < 32 ]
timeToSave = [x for x in timeToSave if x%100!=0]
timeToSave = [x for x in timeToSave if x!=230]
print(timeToSave)

[101, 104, 107, 110, 113, 116, 119, 122, 125, 128, 131, 203, 206, 209, 212, 215, 218, 221, 224, 227, 302, 305, 308, 311, 314, 317, 320, 323, 326, 329, 401, 404, 407, 410, 413, 416, 419, 422, 425, 428, 431, 503, 506, 509, 512, 515, 518, 521, 524, 527, 530, 602, 605, 608, 611, 614, 617, 620, 623, 626, 629, 701, 704, 707, 710, 713, 716, 719, 722, 725, 728, 731, 803, 806, 809, 812, 815, 818, 821, 824, 827, 830, 902, 905, 908, 911, 914, 917, 920, 923, 926, 929, 1001, 1004, 1007, 1010, 1013, 1016, 1019]


In [4]:

print(str(nowDate))

1028
